# Agentics Mini Tutorial

Agentics provides the implementation of **AG**, a powerful datatype that connects
LLMs to Pydantic objects and enables **logical transduction**.

---

## Installation

```bash
!uv pip install agentics-py

In [ ]:
! uv pip install agentics-py


import os
from pathlib import Path
import sys
from getpass import getpass

from dotenv import find_dotenv, load_dotenv

CURRENT_PATH = ""

IN_COLAB = "google.colab" in sys.modules
print("In Colab:", IN_COLAB)


if IN_COLAB:
    CURRENT_PATH = "/content/drive/MyDrive/"
    # Mount your google drive
    from google.colab import drive

    drive.mount("/content/drive")
    from google.colab import userdata

    os.environ["GEMINI_API_KEY"] = getpass("Enter your GEMINI_API_KEY:")
else:

    CURRENT_PATH = os.getcwd()
    load_dotenv(find_dotenv())

if not os.getenv("GEMINI_API_KEY"):
    os.environ["GEMINI_API_KEY"] = getpass("Enter your GEMINI_API_KEY:")

base = Path(CURRENT_PATH)

## Use Agentics as Lists

Agentics objects (`AG`) can be used similarly to Python lists, allowing you to store and manage collections of states. You can append new elements using the `.append()` method, and access all states via the `.states` attribute.

For example, after creating an empty `AG` object, you can add elements:

In [ ]:
from agentics import AG

my_first_agentics = AG()

print("The agentics is empty :", len(my_first_agentics))

## Add elements to the list
my_first_agentics.append("Alfio")
## internally, agentics stores the elements in the attribute states
my_first_agentics.states += ["Naweed", "Junkyuu"]

print("The agentics now has more instances :", len(my_first_agentics))

try:
    print("this triggers an error")
    my_first_agentics = my_first_agentics + my_first_agentics
except:
    my_first_agentics.states = my_first_agentics.states + my_first_agentics.states
    print(
        "This is the right way to concetenate two agentics. Be careful, the states should be instances of the same atype"
    )
    my_first_agentics.pretty_print()

print("Iterating over agentics:")
for state in my_first_agentics:
    print(state)

print("Be careful, the AG itself is not a list :", my_first_agentics)

## Atypes

Agentics supports **typed AGs** using Pydantic models, enabling you to enforce schema validation and structure on the states stored in an AG. This is useful when you want all elements in your AG to follow a specific format or contain certain fields.

To define a typed AG:

1. **Create a Pydantic model** that describes the schema for your states.
2. **Instantiate an AG** with the `atype` parameter set to your Pydantic model.
3. **Add instances** of your model to the AG. Only objects matching the schema will be accepted.

This approach ensures data consistency and allows you to leverage Pydantic's validation features within Agentics workflows.

For example, you can define a `Movie` type and create an AG that only accepts `Movie` instances as its states. See the next cell for a practical demonstration.

In [ ]:
from pydantic import BaseModel
from typing import Optional
from agentics import AG


# Define the Movie Pydantic model for use with Agentics AG
class Movie(BaseModel):
    movie_name: Optional[str] = None
    genre: Optional[str] = None
    description: Optional[str] = None


movies = AG(atype=Movie)
movies.append(Movie(movie_name="La dolce vita"))
movies.pretty_print()

## Extending and Merging AGs
AGs can evolve by adding new fields or combining with other AGs to form richer schemas.

### Add attributes
Use `.add_attribute()` to dynamically extend the schema of an AG.  
This operation mutates the AG in place.

In [ ]:
movies = AG(atype=Movie)
movies.append(Movie(movie_name="La dolce vita"))
movies.pretty_print()

print("adding a new attribute to the type and rebinding the object")
movies = movies.add_attribute(
    "email",
    description="Write an email to tell a fried about this movie",
    slot_type=Optional[str],
)

movies.pretty_print()
print("Note that the AG changed")

### Subtypes
You can project an AG onto a subset of its fields, e.g. `movies("title", "genre")`.  
This creates a new AG without modifying the original.

In [ ]:
movies_subtype = movies("movie_name", "genre")
print("This is a subtype")
movies_subtype.pretty_print()

print("This is the original type.\nNote that the AG didn't change after subtype")
movies.pretty_print()
print("Note that the AG didn't change after subtyping it")

## Import Agentics for Json and CSV 

Agentics AG objects can be easily imported from and exported to CSV and JSONL formats. This enables seamless integration with tabular and structured data workflows.

- **CSV Import/Export:**  
    Use `AG.from_csv("path/to/file.csv")` to create an AG from a CSV file. The schema (`atype`) can be inferred automatically or provided explicitly.
- **JSONL Import/Export:**  
    Use `AG.to_jsonl("path/to/file.jsonl")` to export, and `AG.from_jsonl("path/to/file.jsonl")` to import AG objects in JSON Lines format.

This functionality allows you to move data between Agentics and other tools with minimal effort.

In [ ]:
# Create a new AG object from the provided csv file
movies = AG.from_csv(base / "data/movies.csv", max_rows=3)
movies.pretty_print()

# Note that the atype has been automatically inffered
print("Imported Type", movies.atype)

# Reloading same file by providing atype
movies = AG.from_csv(base / "data/movies.csv", atype=Movie)

# Note that just a subset of the attributes have been imported
print("Provided Type", movies.atype)

# agentics can be exported and imported from jsonl objects
movies.to_jsonl(base / "data/movies.jsonl")
movies = AG.from_jsonl(base / "data/movies.jsonl")

# note this type is different from what imported from csv
print("Imported atype from jsonl: ", movies.atype)

## Logical Transduction

Once an AG is initialized with an atype, Agentics can **transduce** any string of text and/or pydantic object into that type.  If a list of strings is provided, they are processed asynchronously.

## Untyped transduction

If no target atype is provided, transduction works as a regular llm call, where the input text or pydantic object is given to the LLM and the output is the LLM response. In this use case, agentics provides an off the shelp **async scale-out framework for LLM calls**. 

Note that no AType is specified, the output of transduction is alist of strings. So it is not recommended to use this notation for transduction algebra. In addition, Unconstrained trnasduction tends to me less efficient as it requires the LLM to guess the type of output required, often resulting in verbose and unecessary information . 

In [ ]:
import time

questions = [
    "What are the benefits of using Agentic AI for data workflows?",
    "Will AI improve working conditions for the middle class?",
    "How can Agentic AI enhance decision-making in finance?",
    # "What risks should companies consider when adopting AI agents?",
    # "Can AG objects integrate with existing data pipelines?",
    # "Who won the latest FIFA worldcup",
]
start = time.time()
answers = await (AG() << questions)
end = time.time()

for question, answer in zip(questions, answers):
    print(f"Question: {question}\nAnswer{answer}\n")
print(f"Uncostrained transduction done in {end-start} seconds")

### Transduction into Atype

You can define a target schema with Pydantic (e.g., `Answer`) and transduce text into it.  
The LLM output is parsed and validated into the fields `answer`, `justification`, and `confidence`.  
Note that the output is more clean and organized, and the time required to execute the transduction is one order of magnitude lower. 

In [ ]:
# Define a Pydantic model for a structured answer
class Answer(BaseModel):
    # The main response text
    answer: Optional[str] = None
    # An explanation or reasoning behind the answer
    justification: Optional[str] = None
    # A numeric confidence score (e.g. from 0.0 to 1.0)
    confidence: Optional[float] = None


# Transduce a natural language question into the structured Answer schema
start = time.time()
answers = await (AG(atype=Answer) << questions)
end = time.time()
print(f"Typed transduction done in {end-start} seconds")
answers.pretty_print()

### Transduction Between AGs

You can control transduction more precisely by converting **from one AG to another**:
- The **source AG** provides the input states (rendered via the prompt).
- The **target AG** defines the output schema and validation.
- Agentics renders each source state → sends it to the LLM → parses into the target type.

This pattern is ideal when you want consistent, structured outputs from heterogeneous inputs while keeping prompts and schema separate.

### Transduction Between AGs  
Here we convert product reviews (`ProductReview`) into sentiment summaries (`SentimentSummary`).  
The source AG provides the reviews, and the target AG enforces structured outputs (positive/neutral/negative with a reason).  

In [ ]:
from typing import Optional, Literal
from pydantic import BaseModel
from agentics import AG


# Source schema: product reviews
class ProductReview(BaseModel):
    reviewer: Optional[str] = None
    text: Optional[str] = None
    stars: Optional[int] = None


# Target schema: summarized sentiment
class SentimentSummary(BaseModel):
    customer_sentiment: Optional[Literal["positive", "neutral", "negative"]] = None
    reason: Optional[str] = None


# Example reviews
reviews = [
    ProductReview(
        reviewer="Alice", text="Excellent quality and fast delivery!", stars=5
    ),
    ProductReview(reviewer="Bob", text="Okay, but packaging was damaged", stars=3),
    ProductReview(reviewer="Carol", text="Terrible, broke after one use", stars=1),
]

# Create source and target AGs
source = AG(atype=ProductReview, states=reviews)
target = AG(atype=SentimentSummary, provide_explanations=True)

# Transduce reviews into sentiment summaries
sentiments = await (target << source)

sentiments.pretty_print()


### Self-Transduction  

You can transduce within the same AG type by selecting different subsets of fields.  
This is useful for projecting, comparing, or enriching dataframes and state graphs without changing the original AG.  

In [ ]:
movies = AG.from_csv(base / "data/movies.csv", atype=Movie)
movies.filter_states(start=10, end=20)

self_transductions = await movies.self_transduction(
    ["movie_name", "description"], ["genre"]
)
print(self_transductions.pretty_print())

### Customizing Transduction  

You can fine-tune how logical transduction works by configuring:  

- **LLMs** – choose the underlying language model to run the transduction.  
- **Instructions** – add task-specific guidance for the LLM.  
- **Prompt Templates** – control how inputs are rendered into prompts.  
- **Few-Shot Examples** – provide examples to steer the model’s behavior.  
- **Verbose Options** – enable detailed logging and debug outputs.  

#### Task instructions
The example below illustrate how to provide a llm and task specific instructions to transduction

In [ ]:
questions_answering_ag = AG(
    atype=Answer, llm=AG.get_llm_provider("watsonx"), instructions="Answer in italian"
)

print((await (questions_answering_ag << questions)).pretty_print())

#### Prompt templates

Prompt templates enable greater customization of your transductions by providing a langchain style abstraction to render pydantic objects into input prompts for the agent. 



In [ ]:
questions_answering_ag = AG(atype=Answer)

dow_jones_data = AG.from_csv("data/dow_jones.csv")
dow_jones_data = dow_jones_data.get_random_sample(0.002)
dow_jones_data.prompt_template = "what happened to the financial markets in {date}?"
answers = await (questions_answering_ag << dow_jones_data)
print(answers.pretty_print())

### Few-Shot Learning

Agentics natively supports **few-shot examples**: you can preload the **target AG** with
gold states (examples of the desired output). During transduction, these examples steer
the LLM toward consistent labels/structures.

Below, we load movies from CSV, manually seed the first 10 labels as few-shots, and then
transduce the rest.

In [ ]:
# Zero Shots= all output states are blank
movies = AG.from_csv(base / "data/movies.csv", atype=Movie)
movies.filter_states(start=10, end=20)
self_transductions = await movies.self_transduction(
    ["movie_name", "description"], ["genre"]
)
print("this is zero shot")
self_transductions.pretty_print()

In [ ]:
# Few shots = the first 10 states are used as examples
movies = AG.from_csv(base / "data/movies.csv", atype=Movie)
movies.filter_states(start=0, end=20)
self_transductions = await movies.self_transduction(
    ["movie_name", "description"], ["genre"]
)

# printing only transduced states
movies.filter_states(start=10, end=20)
print("this is the output with 10 shots")
self_transductions.pretty_print()

## Tool Usage  

Agentics integrates seamlessly with the **MCP ecosystem**, allowing AGs to call external tools during transduction.  In addition to that, they also allows the use of CrewAI tools, as the underlying transduction framework is currently based on crewAI agents. 

This makes it easy to fetch, process, or enrich data dynamically while keeping results structured. 

In the following example we illustrate the use of Duck Duck Go search to improve the information gathering of historical market data. 

In [ ]:
from crewai.tools import tool
from ddgs import DDGS


## Define a Crew AI tool to get news for a given date using the DDGS search engine
@tool("web_search")
def web_search(query: str) -> str:
    """Fetch web search results for the given query using DDGS."""
    return str(DDGS().text(query, max_results=10))


questions_answering_ag.verbose_agent = True
questions_answering_ag.tools = [web_search]
dow_jones_data.filter_states(end=1)
answers = await (questions_answering_ag << dow_jones_data)
print(answers.pretty_print())